In [1]:
from pathlib import Path
import pandas as pd
ROOT = Path('../..')
ROOT.resolve()

PosixPath('/Users/lukestrange/Code/housing')

In [2]:
d = pd.read_excel(ROOT / 'raw/additional-dwellings/LT122.ods', engine='odf', sheet_name='LT122', header=3, na_values=['[p]', '[r]', '[x]'])

In [3]:
import re
# Drop all the rows that don't have a current ONS code
d = d[~d['CurrentONS code'].isna()]

# drop unused columns
d.drop(columns=['DCLG code', 'FormerONS code', 'Authority Data'], inplace=True)

# Rename geography column code and set as index
d.rename(columns={'CurrentONS code': 'geography_code'}, inplace=True)
d.set_index('geography_code', inplace=True)

# Remove anything that isn't actually a date from the date column titles
d.columns = [re.match(r'\d{4}-\d{2}', col).group() for col in d.columns]
d = d.round(0)
d

,2001-02,2002-03,2003-04,2004-05,2005-06,2006-07,2007-08,2008-09,2009-10,2010-11,...,2013-14,2014-15,2015-16,2016-17,2017-18,2018-19,2019-20,2020-21,2021-22,2022-23
geography_code,,,,,,,,,,,,,,,,,,,,,
E06000022,270.0,264.0,301.0,151.0,172.0,258.0,497.0,378.0,346.0,241.0,...,557.0,640.0,818.0,880,1254,1049,1176.0,773.0,552.0,490.0
E06000055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,507.0,675.0,...,1030.0,862.0,998.0,1288.5,1383.5,1392.5,1060.0,1232.0,1019.0,1465.0
E06000008,-74.0,263.0,639.0,-94.0,594.0,230.0,619.0,82.0,336.0,372.0,...,246.0,235.0,115.0,159.1,299.1,505.1,323.0,461.0,440.0,553.0
E06000009,160.0,55.0,242.0,241.0,240.0,174.0,357.0,311.0,-1.0,181.0,...,144.0,294.0,447.0,51,473,565,532.0,358.0,267.0,233.0
E06000028,764.0,920.0,1103.0,1036.0,900.0,1029.0,1474.0,1173.0,562.0,432.0,...,392.0,962.0,721.0,579.487666,633.487666,657.487666,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E07000235,122.0,370.0,202.0,210.0,180.0,299.0,321.0,271.0,237.0,151.0,...,170.0,244.0,309.0,348.6,440.6,562.6,378.0,449.0,407.0,227.0
E07000236,248.0,290.0,422.0,325.0,292.0,482.0,245.0,109.0,178.0,130.0,...,174.0,341.0,187.0,188.8,389.8,395.8,285.0,205.0,127.0,103.0
E07000237,3.0,250.0,219.0,284.0,476.0,522.0,432.0,476.0,386.0,240.0,...,288.0,468.0,617.0,477.9,255.9,239.9,222.0,352.0,414.0,273.0


In [4]:
d = d.reset_index().melt(id_vars='geography_code', value_vars=d.columns, var_name='date', value_name='value')
d.to_parquet(ROOT / 'data/additional-dwellings/parquet/by_local_authority.parquet', index=False)

In [5]:
d['Measure'] = 'net_additional_dwellings'
d.to_csv(ROOT / 'data/additional-dwellings/csv/by_local_authority.csv', index=False)